In [31]:
import pandas as pd
from scipy.sparse import load_npz

df_preprocessed_general = load_npz('./ds_salaries_GeneralPreprocessing.npz')
target = pd.read_csv("./ds_salaries_target.csv")

# Baseline Model (Linear Regression)

In [32]:
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import cross_val_score, KFold
import numpy as np

estimator = LinearRegression()

steps = [('estimator', estimator)]

pipeline = Pipeline(steps)

kfold = KFold(n_splits=5, shuffle=True, random_state=42)

r2_scores = cross_val_score(pipeline, df_preprocessed_general, target, cv=kfold, scoring='r2')
mse_scores = -cross_val_score(pipeline, df_preprocessed_general, target, cv=kfold, scoring='neg_mean_squared_error')

print("Variante1")
print(np.mean(r2_scores))
print(np.mean(mse_scores))


Variante1
0.39823610106618296
2390630578.4489965


# init Model

In [34]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR

estimator = GradientBoostingRegressor()

# Execute pipeline and evaluate

In [35]:
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import cross_val_score, KFold
import numpy as np

steps = [('estimator', estimator)]

pipeline = Pipeline(steps)

kfold = KFold(n_splits=5, shuffle=True, random_state = 42)

r2_scores = cross_val_score(pipeline, df_preprocessed_general, target, cv=kfold, scoring='r2')
mse_scores = -cross_val_score(pipeline, df_preprocessed_general, target, cv=kfold, scoring='neg_mean_squared_error')


print(np.mean(r2_scores))
print(np.mean(mse_scores))

C:\Users\timbl\AppData\Roaming\Python\Python310\site-packages\sklearn\ensemble\_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\timbl\AppData\Roaming\Python\Python310\site-packages\sklearn\ensemble\_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\timbl\AppData\Roaming\Python\Python310\site-packages\sklearn\ensemble\_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\timbl\AppData\Roaming\Python\Python310\site-packages\sklearn\ensemble\_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array w

0.42064831097103106
2301918061.949946


In [11]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold 

# Modell initialisieren
regressor = GradientBoostingRegressor()

# Parameter Grid für Grid Search definieren
parameters = {
    'n_estimators': [100, 500, 1000],
    'learning_rate': [0.01, 0.1, 1],
    'max_depth': [2, 3, 5]
}

k_10_fold_cv = KFold(n_splits=10, shuffle=True, random_state=42)

# Grid Search initialisieren
grid_search_estimator = GridSearchCV(regressor, parameters, scoring='r2', cv=k_10_fold_cv, return_train_score=False)

grid_search_estimator.fit(df_preprocessed_general,target)

#results = pd.DataFrame(grid_search_estimator.cv_results_)
    
# print the best parameter setting
print("best score is {} with params {}".format(grid_search_estimator.best_score_, grid_search_estimator.best_params_))


best score is 0.4331969080281256 with params {'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 1000}


In [15]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold 

# Modell initialisieren
regressor = GradientBoostingRegressor()

# Parameter Grid für Grid Search definieren
parameters = {
    'n_estimators': [100, 500, 1000],
    'learning_rate': [0.01, 0.1, 1],
    'max_depth': [2, 3, 5],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

k_10_fold_cv = KFold(n_splits=10, shuffle=True, random_state=42)

# Grid Search initialisieren
grid_search_estimator = GridSearchCV(regressor, parameters, scoring='r2', cv=k_10_fold_cv, n_jobs=-1, return_train_score=False)

grid_search_estimator.fit(df_preprocessed_general,target)

#results = pd.DataFrame(grid_search_estimator.cv_results_)
    
# print the best parameter setting
print("best score is {} with params {}".format(grid_search_estimator.best_score_, grid_search_estimator.best_params_))


best score is 0.43718198300279154 with params {'learning_rate': 0.1, 'max_depth': 3, 'max_features': 'auto', 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 500}


In [8]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold 

# Modell initialisieren
regressor = GradientBoostingRegressor()

# Parameter Grid für Grid Search definieren
parameters = {
    'learning_rate': [0.025, 0.05, 0.1, 0.2, 0.3],
    'max_depth': [2, 3, 5, 7, 10, 3800],
    'min_samples_split': [2, 5, 10, 20],
    'max_features': [0.25, 1.0, 'sqrt', 'log2'],
    'subsample': [0.15, 0.5, 0.75, 1.0]
}

k_10_fold_cv = KFold(n_splits=10, shuffle=True, random_state=42)

# Grid Search initialisieren
grid_search_estimator = GridSearchCV(regressor, parameters, scoring='r2', cv=k_10_fold_cv, n_jobs=-1, return_train_score=False)

grid_search_estimator.fit(df_preprocessed_general,target)

#results = pd.DataFrame(grid_search_estimator.cv_results_)
    
# print the best parameter setting
print("best score is {} with params {}".format(grid_search_estimator.best_score_, grid_search_estimator.best_params_))

best score is 0.4379085157268688 with params {'learning_rate': 0.1, 'max_depth': 10, 'max_features': 'sqrt', 'min_samples_split': 20, 'subsample': 1.0}


In [9]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold 

# Modell initialisieren
regressor = GradientBoostingRegressor()

# Parameter Grid für Grid Search definieren
parameters = {
    'learning_rate': [0.1],
    'max_depth': [10],
    'min_samples_split': [20],
    'max_features': ['sqrt', 'log2'],
    'subsample': [1.0]
}

k_10_fold_cv = KFold(n_splits=10, shuffle=True, random_state=42)

# Grid Search initialisieren
grid_search_estimator = GridSearchCV(regressor, parameters, scoring='r2', cv=k_10_fold_cv, return_train_score=False)

grid_search_estimator.fit(df_preprocessed_general,target)

#results = pd.DataFrame(grid_search_estimator.cv_results_)
    
# print the best parameter setting
print("best score is {} with params {}".format(grid_search_estimator.best_score_, grid_search_estimator.best_params_))

best score is 0.43817034079751876 with params {'learning_rate': 0.1, 'max_depth': 10, 'max_features': 'log2', 'min_samples_split': 20, 'subsample': 1.0}
